In [2]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr


In [3]:
data = pd.read_excel(r"all_samples_clean.xlsx", sheet_name="Sheet1")
data = data.drop(['BASIC_entity_name', 'BASIC_year'], axis=1)

In [4]:
# 计算Spearman相关系数
spearman_corr_matrix, _ = spearmanr(data)

#print("Spearman相关系数矩阵：")
#print(spearman_corr_matrix)

In [6]:
spearman_corr_matrix = pd.DataFrame(spearman_corr_matrix)
spearman_corr_matrix.columns = data.columns

In [8]:
#spearman_corr_matrix.to_excel(r"spearman.xlsx", sheet_name="Sheet1")

In [7]:
spearman_corr_matrix  = pd.DataFrame(spearman_corr_matrix)
#spearman_corr_matrix.to_excel(r"all_samples_spearman.xlsx", sheet_name="Sheet1")

In [8]:
spearman_corr_matrix.columns=data.columns

In [10]:
spearman_corr_matrix.index=data.columns

In [21]:
from scipy.stats import chi2_contingency

In [22]:
cramer_v_dict = {}

for i in data.columns:
    cramer_v_dict[i] = {}
    for j in data.columns:
        # 创建列联表
        cross_tab = pd.crosstab(data[i], data[j])
        # 计算Cramer's V
        chi2, p, dof, expected = chi2_contingency(cross_tab)
        n = cross_tab.sum().sum()
        cramer_v = np.sqrt(chi2 / (n * (min(cross_tab.shape) - 1)))
        cramer_v_dict[i][j] = cramer_v

# 将结果转换为DataFrame
cramer_v_df = pd.DataFrame(cramer_v_dict)

In [44]:
cramer_v_df.to_excel(r"cramer_v.xlsx", sheet_name="Sheet1")

In [37]:
# 计算Cramer's V的函数
def calculate_cramer_v(cross_tab):
    chi2 = chi2_contingency(cross_tab)[0]
    n = cross_tab.sum().sum()
    cramer_v = np.sqrt(chi2 / (n * (min(cross_tab.shape) - 1)))
    return cramer_v



In [42]:
#用字典存储最大Cramer's V值及其对应的行名（不包括值为1的情况）
max_cramer_v_values = {}
for col in cramer_v_df.columns:
    # 获取所有小于1的值，并排除值为1的情况
    filtered_values = cramer_v_df[col][(cramer_v_df[col] < 1) & (cramer_v_df[col] == cramer_v_df[col].max())]
    if not filtered_values.empty:
        max_value = filtered_values
        max_row_name = filtered_values.index[0]
        max_cramer_v_values[col] = {'max_value': max_value, 'row_name': max_row_name}


In [43]:
# 将字典转换为DataFrame
max_cramer_v_df = pd.DataFrame.from_dict(max_cramer_v_values, orient='index')
#保存为excel
#max_cramer_v_df.to_excel(r"max_row_Cramer's V.xlsx", sheet_name="Sheet1")
# 打印包含最大Cramer's V值及其对应行名的DataFrame（不包括值为1的情况）
print(max_cramer_v_df)

                                                                   max_value  \
COSTAB_ACCRUALS            COSTAB_ACCRUALS    1.0
Name: COSTAB_ACCRUALS, ...   
COSTAB_PROFITGROWTH        COSTAB_PROFITGROWTH    1.0
Name: COSTAB_PROFIT...   
DEBTAB_DEBTSTRUCTURE       DEBTAB_DEBTSTRUCTURE    0.999601
Name: DEBTAB_...   
DEBTAB_FINANCINGSTRUCTURE  DEBTAB_FINANCINGSTRUCTURE    0.999808
Name: DE...   
DILEMMAAB_OPCASH           DILEMMAAB_OPCASH    1.0
Name: DILEMMAAB_OPCASH...   

                                            row_name  
COSTAB_ACCRUALS                      COSTAB_ACCRUALS  
COSTAB_PROFITGROWTH              COSTAB_PROFITGROWTH  
DEBTAB_DEBTSTRUCTURE            DEBTAB_DEBTSTRUCTURE  
DEBTAB_FINANCINGSTRUCTURE  DEBTAB_FINANCINGSTRUCTURE  
DILEMMAAB_OPCASH                    DILEMMAAB_OPCASH  
